In [1]:
# Code by Zangror: 
#    https://blogs.sas.com/content/sgf/2016/10/12/the-monty-hall-paradox-sas-vs-python/
#    https://0bin.net/paste/lLtXYs+6YCDzFC7T#Ez6Q4F9R0AlT1WdzUk7GJ7DmmQt6S6O2QJvJs9vfPFD
# Here for reference and benchmarking purposes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
%%timeit
# Create the data frame
df = pd.DataFrame.from_dict({
    'door_with_prize': np.random.randint(1, 4, 100000),
    'door_choice': np.random.randint(1, 4, 100000)
})
 
# Reveal the door
def reveal_door_function(row):
    choices = tuple({1, 2, 3} - {row['door_with_prize'], row['door_choice']})
    return int(np.random.choice(choices))
 
df['revealed_door'] = df.apply(reveal_door_function, axis=1)
 
# Make the not switch strategy
df['never_switch_strategy'] = df['door_choice']
# Make the switch strategy
def switch_function(row):
    return tuple({1, 2, 3} - {row['revealed_door'], row['door_choice']})[0]
df['switch_strategy'] = df.apply(switch_function, axis=1)
 
# Now compute when the switch or the non switch strategy win
df['never_switch_strategy_win'] = (df['never_switch_strategy'] == df['door_with_prize']).astype(int)
df['switch_strategy_win'] = (df['switch_strategy'] == df['door_with_prize']).astype(int)
 
# Do a cumulative sum of the strategies
df['cum_never_switch_win'] = df['never_switch_strategy_win'].cumsum()
df['cum_switch_win'] = df['switch_strategy_win'].cumsum()
 
# Divide by the number of observations for a win percentage
df['percentage_never_switch_win'] = df['cum_never_switch_win'] / (df.index + 1)
df['percentage_switch_win'] = df['cum_switch_win'] / (df.index + 1)

8.57 s ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
axis = df[['percentage_switch_win', 'percentage_never_switch_win']].plot.area(stacked=False)
axis.set_xlabel("Iterations", size=14)
axis.set_ylabel("Win Pct",  size=14)
plt.title("Cumulative chances of winning on Let's Make a Deal", size=16)
plt.grid(True)